In [2]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import os
import matplotlib.pyplot as plt
import cv2
from torch.utils.data import Dataset, DataLoader
import numpy as np

class ShapesDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.landmarks_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,
                                self.landmarks_frame.iloc[idx, 0])
        image = io.imread(img_name)
        landmarks = self.landmarks_frame.iloc[idx, 1:].as_matrix()
        landmarks = landmarks.astype('float').reshape(-1, 2)
        sample = {'image': image, 'landmarks': landmarks}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [3]:
dic = dict((c, i) for i, c in enumerate(os.listdir('Shapes/Train/')))
dic

{'Circle': 2, 'Hexagon': 0, 'Pentagon': 3, 'Rectangle': 4, 'Triangle': 1}

In [4]:
def load_images_from_folder(folder):
    images = []
    shape=[]
    for filename in os.listdir(folder):
        for image in os.listdir(folder+str(filename)):
            img = cv2.imread(os.path.join(folder+str(filename),image))
            if img is not None:
                images.append(img.transpose((2, 0, 1)) )
                shape.append(dic[str(filename)])
    return images,shape    

In [5]:
class ShapesDataset(Dataset):

    def __init__(self,root_dir, transform=None):
        self.image=[]
        self.shape=[]
        self.root_dir = root_dir
        self.transform = transform
        dic = dict((c, i) for i, c in enumerate(os.listdir(self.root_dir)))
        for filename in os.listdir(self.root_dir):
            for image in os.listdir(self.root_dir+str(filename)):
                img = cv2.imread(os.path.join(self.root_dir+str(filename),image))
                if img is not None:
                    self.image.append(img.transpose((2,0,1)))
                    self.shape.append(dic[str(filename)])

    def __len__(self):
        return len(self.shape)

    def __getitem__(self, idx):
        #if self.transform:
            #sample = self.transform(sample) 
        self.image=np.array(self.image)
        return torch.from_numpy(self.image[idx]/255.0),(self.shape[idx])

In [6]:
i,s=load_images_from_folder('Shapes/Test/')
# c=0
# for j in range(len(s)):
#     if(s[j]=='Circle'):
#         c+=1
#print(i[2000],s[2000]) 
#type(s)
np.array(i).shape

(3857, 3, 28, 28)

In [12]:
# Training settings
batch_size = 16

# Dataset
train_dataset = ShapesDataset(root_dir='Shapes/Train/',
                               transform=transforms.ToTensor())

test_dataset = ShapesDataset(root_dir='Shapes/Test/',
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [11]:
np.array(train_dataset.__getitem__(2)).shape()

TypeError: 'tuple' object is not callable

In [13]:
class Branched_CNN(nn.Module):

    def __init__(self, in_channels):
        super(Branched_CNN, self).__init__()
        self.branch1x1 = nn.Conv2d(in_channels, 16, kernel_size=1)

        self.branch5x5_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch5x5_2 = nn.Conv2d(16, 24, kernel_size=5, padding=2)

        self.branch3x3_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch3x3_2 = nn.Conv2d(16, 24, kernel_size=3, padding=1)
        self.branch3x3_3 = nn.Conv2d(24, 24, kernel_size=3, padding=1)

        self.branch_pool = nn.Conv2d(in_channels, 24, kernel_size=1)

    def forward(self, x):
        branch1x1 = self.branch1x1(x.float())

        branch5x5 = self.branch5x5_1(x.float())
        branch5x5 = self.branch5x5_2(branch5x5)

        branch3x3 = self.branch3x3_1(x.float())
        branch3x3 = self.branch3x3_2(branch3x3)
        branch3x3 = self.branch3x3_3(branch3x3)

        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch5x5, branch3x3, branch_pool]
        return torch.cat(outputs, 1)

In [78]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3)
        self.conv2 = nn.Conv2d(88, 16, kernel_size=3)

        self.network1 = Branched_CNN(in_channels=64)
        self.network2 = Branched_CNN(in_channels=16)

        self.mp = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(2200,1600)
        self.fc2 = nn.Linear(1600,800)
        self.fc3 = nn.Linear(800,128)
        self.fc4 = nn.Linear(128,16)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x.float())))
       
        x = self.network1(x.float())
        x = F.relu(self.mp(self.conv2(x.float())))
        x = self.network2(x.float())
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        return F.log_softmax(x)

In [79]:
model = Net()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [80]:
model.parameters

<bound method Module.parameters of Net(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(88, 16, kernel_size=(3, 3), stride=(1, 1))
  (network1): Branched_CNN(
    (branch1x1): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
    (branch5x5_1): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
    (branch5x5_2): Conv2d(16, 24, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (branch3x3_1): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
    (branch3x3_2): Conv2d(16, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (branch3x3_3): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (branch_pool): Conv2d(64, 24, kernel_size=(1, 1), stride=(1, 1))
  )
  (network2): Branched_CNN(
    (branch1x1): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1))
    (branch5x5_1): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1))
    (branch5x5_2): Conv2d(16, 24, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (branch3x3_1): Conv2d(16,

In [81]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        #print(target," :",output)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

In [82]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).data[0]
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [83]:
for epoch in range(1, 5):
    train(epoch)
    test()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/17168 (0%)]	Loss: 2.740190
Train Epoch: 1 [160/17168 (1%)]	Loss: 2.615175
Train Epoch: 1 [320/17168 (2%)]	Loss: 2.369420
Train Epoch: 1 [480/17168 (3%)]	Loss: 1.652780
Train Epoch: 1 [640/17168 (4%)]	Loss: 1.717301
Train Epoch: 1 [800/17168 (5%)]	Loss: 1.574546
Train Epoch: 1 [960/17168 (6%)]	Loss: 1.458035
Train Epoch: 1 [1120/17168 (7%)]	Loss: 1.603423
Train Epoch: 1 [1280/17168 (7%)]	Loss: 1.770942
Train Epoch: 1 [1440/17168 (8%)]	Loss: 1.758655
Train Epoch: 1 [1600/17168 (9%)]	Loss: 1.556695
Train Epoch: 1 [1760/17168 (10%)]	Loss: 1.673056
Train Epoch: 1 [1920/17168 (11%)]	Loss: 1.785106
Train Epoch: 1 [2080/17168 (12%)]	Loss: 1.548995
Train Epoch: 1 [2240/17168 (13%)]	Loss: 1.576725
Train Epoch: 1 [2400/17168 (14%)]	Loss: 1.578160
Train Epoch: 1 [2560/17168 (15%)]	Loss: 1.709205
Train Epoch: 1 [2720/17168 (16%)]	Loss: 1.600527
Train Epoch: 1 [2880/17168 (17%)]	Loss: 1.547848
Train Epoch: 1 [3040/17168 (18%)]	Loss: 1.672262
Train Epoch: 1 [3200/17168 (19%)]	Loss: 

Train Epoch: 2 [9280/17168 (54%)]	Loss: 1.255319
Train Epoch: 2 [9440/17168 (55%)]	Loss: 1.210909
Train Epoch: 2 [9600/17168 (56%)]	Loss: 1.556051
Train Epoch: 2 [9760/17168 (57%)]	Loss: 1.299196
Train Epoch: 2 [9920/17168 (58%)]	Loss: 1.212731
Train Epoch: 2 [10080/17168 (59%)]	Loss: 1.359152
Train Epoch: 2 [10240/17168 (60%)]	Loss: 1.365412
Train Epoch: 2 [10400/17168 (61%)]	Loss: 1.339549
Train Epoch: 2 [10560/17168 (62%)]	Loss: 1.193452
Train Epoch: 2 [10720/17168 (62%)]	Loss: 1.318867
Train Epoch: 2 [10880/17168 (63%)]	Loss: 1.191248
Train Epoch: 2 [11040/17168 (64%)]	Loss: 1.255425
Train Epoch: 2 [11200/17168 (65%)]	Loss: 0.991801
Train Epoch: 2 [11360/17168 (66%)]	Loss: 1.363989
Train Epoch: 2 [11520/17168 (67%)]	Loss: 1.121948
Train Epoch: 2 [11680/17168 (68%)]	Loss: 1.162571
Train Epoch: 2 [11840/17168 (69%)]	Loss: 1.133628
Train Epoch: 2 [12000/17168 (70%)]	Loss: 1.292998
Train Epoch: 2 [12160/17168 (71%)]	Loss: 1.266446
Train Epoch: 2 [12320/17168 (72%)]	Loss: 1.292065
Train

Train Epoch: 4 [960/17168 (6%)]	Loss: 1.506382
Train Epoch: 4 [1120/17168 (7%)]	Loss: 1.208722
Train Epoch: 4 [1280/17168 (7%)]	Loss: 1.318329
Train Epoch: 4 [1440/17168 (8%)]	Loss: 1.384972
Train Epoch: 4 [1600/17168 (9%)]	Loss: 1.079344
Train Epoch: 4 [1760/17168 (10%)]	Loss: 1.359560
Train Epoch: 4 [1920/17168 (11%)]	Loss: 1.523851
Train Epoch: 4 [2080/17168 (12%)]	Loss: 1.369179
Train Epoch: 4 [2240/17168 (13%)]	Loss: 1.058241
Train Epoch: 4 [2400/17168 (14%)]	Loss: 1.211343
Train Epoch: 4 [2560/17168 (15%)]	Loss: 1.103756
Train Epoch: 4 [2720/17168 (16%)]	Loss: 1.263960
Train Epoch: 4 [2880/17168 (17%)]	Loss: 1.325978
Train Epoch: 4 [3040/17168 (18%)]	Loss: 1.286080
Train Epoch: 4 [3200/17168 (19%)]	Loss: 1.237329
Train Epoch: 4 [3360/17168 (20%)]	Loss: 1.317142
Train Epoch: 4 [3520/17168 (21%)]	Loss: 1.543058
Train Epoch: 4 [3680/17168 (21%)]	Loss: 1.382810
Train Epoch: 4 [3840/17168 (22%)]	Loss: 1.444353
Train Epoch: 4 [4000/17168 (23%)]	Loss: 1.165121
Train Epoch: 4 [4160/17168